In [ ]:
import numpy as np
import pandas as pd

In [ ]:
from scipy.optimize import curve_fit

In [ ]:
from itertools import product

In [ ]:
from commons import smoothen, lse

In [ ]:
n_iterations = 20

# Preparations

## Loading the data

We start by loading the $N_i(t)$, while smoothening them :

In [ ]:
Nt = smoothen(np.load("curves_raw.npy"), 10)

Let's also calculate already the derivatives $\frac{\Delta N_i(t)}{\Delta t} = \Delta N_i(t) = N_i(t+1) - N_i(t)$

In [ ]:
dNdt = Nt[..., 1:] - Nt[..., :-1]

and $\rho_i(t) = \frac{\Delta N_i(t)}{N_i(t)}$

In [ ]:
rho = dNdt / Nt[..., :-1]

Let's get the number of points consistent between $N_i(t)$ and the derivatives :

In [ ]:
Nt = Nt[..., :-1]

## Dimensions

We also get the dimensions

In [ ]:
n_plates, n_rows, n_columns, n_points = Nt.shape
plates, rows, columns, points = map(np.arange, Nt.shape)

## Initial $\alpha_i(t)$ and $\epsilon_k(t)$ parameter values

The density-specific $\rho_i(t) = \alpha_i(t) \; \epsilon_k(t) \; \phi(N_i(t))$ model requires us to provide $\alpha_i(t)$ and $\epsilon_k(t)$ values, which we load here :

In [ ]:
alpha = np.load("alpha/computed.npy")[..., :-10]

In [ ]:
epsilon = pd.read_csv("epsilon/alpha-epsilon_k.csv")
epsilon.index = pd.MultiIndex.from_frame(epsilon[["plate", "time point"]])
epsilon = epsilon[[ f"epsilon {k+1}" for k in range(16) ]]

As we will recompute these values iteratively, along with the parameters that lead for those values, so here are some initial parameter values :

In [ ]:
alphas = pd.read_csv("alpha/params.csv")
alphas.index = pd.MultiIndex.from_frame(alphas[["plate", "row", "column"]])
alphas = alphas[["r0 i", "c i", "m i"]]

The $r_0$ and $m$ parameters are global

In [ ]:
r0_m = pd.DataFrame(index = pd.Index(plates, name = "plate"))

r0_m["r0"] = alphas["r0 i"].unique()
r0_m["m"] = alphas["m i"].unique()

while the $c_i$ parameters are population-specific

In [ ]:
c_i = alphas["c i"]
c_i.index = alphas.index

## Layers

The $\epsilon_k(t)$ are organised in $k = 16$ layers, defined as equidistant points compared to their closest grid border.
We create here 16 matrices where for each layer the respective coordinates in the grid are set to 1 (0 otherwise) :

In [ ]:
layers = np.zeros((16, n_rows, n_columns))

for i in range(16):
    layers[i, i, i:n_columns-i] = 1
    layers[i, i:n_rows-i, i] = 1
    layers[i, n_rows-i-1, i:n_columns-i] = 1
    layers[i, i:n_rows-i, n_columns-i-1] = 1

Convert a coordinate to the corresponding $\epsilon_k$ :

In [ ]:
c2l = sum( (i+1) * l for i, l in enumerate(layers) ).astype(int)

# $\hat\rho_i(t) = \alpha_i(t) \; \epsilon_k(t) \; \phi(N_i(t))$

Unlike the $\hat\rho_i(t) = \alpha_i(t) \; \epsilon(t)$ and $\hat\rho_i(t) = \alpha_i(t) \; \epsilon_k(t)$ models, this model adds nutrient information to the growth ; it thus becomes a system of two differential equations :
$$ \rho_i(t) = \alpha_i(t) \; \epsilon_k(t) \; \phi(s(t, x_i)) $$

$$ \dot s = -\nu \dot N = -\nu \; \alpha_i(t) \; \epsilon_k(t) \; s(i, x_i) \; N_i(t) $$

Let's solve the second equation as :
$$ s(t, x_i) = s_0 e^{-\int_0^t \nu \; \alpha_i(t') \; \epsilon_k(t') \; N_i(t) \; dt'} $$

We thus obtain the following function :
$$ \rho_i(t, x_i, N_i) = \alpha_i(t) \; \epsilon_i(t) \; s_0 e^{-\int_0^t \nu \; \alpha_i(t') \; \epsilon_k(t', x_i) \; N_i(t') \; dt'} $$

The general idea with this model is thus that we first find a value for $\epsilon_k(t)$ for every time point $t$ and layer $k$ ; then second, we find a value $\nu$.
Then, if we recompute new optimal parameters for $\alpha_i(t)$, and then for $\epsilon_k(t)$ and $\nu$ again, we obtain an iterative process :

## Initial $\nu$

We start by obtaining initial values for the $\nu$ parameter, as initial $\alpha_i(t)$ and $\epsilon_k(t)$ values are already obtained from the $\hat\rho_i(t) = \alpha_i(t) \; \epsilon_k(t)$ model, which corresponds to the case where $\nu = 0$ :

In [ ]:
nus = pd.DataFrame(
    data    = np.empty((n_plates, 1)),
    columns = ["nu"],
    index   = pd.Index(plates, name = "plate")
)

In [ ]:
for p in plates:
    integral = -(
            alpha[p].reshape((-1, n_points))
        *   epsilon.loc[p].dot(layers.reshape((16, -1))).T.values
        *   Nt[p].reshape((-1, n_points))
    ).cumsum(axis = 1)
    
    nus.loc[p] = curve_fit(
        f     = lambda _, nu:
                    (
                        alpha[p].reshape((-1, n_points))
                    *   epsilon.loc[p].dot(layers.reshape((16, -1))).T.values
                    *   np.exp(nu * integral)
                    ).reshape(-1),
        xdata = None,
        ydata = rho[p].reshape(-1),
        p0    = 1e-6
    )[0]

## Full model

We then perform the iteration 20 times (or less, if we reached some local minimum) :

In [ ]:
previous_score = np.inf
for it in range(1, n_iterations+1):
    print("iteration", it)
    _epsilon = epsilon.copy()

    print("\tepsilon")
    previous_epsilon = epsilon.copy()
    for p in plates:
        t = 0
        integral = np.exp(-(
                nus.loc[p, "nu"]
                *   alpha[p].reshape((-1, n_points))
                *   epsilon.loc[p].dot(layers.reshape((16, -1))).T.values
                *   Nt[p].reshape((-1, n_points))
            ).cumsum(axis = 1))

        _epsilon.loc[p, t] = curve_fit(
            f     = lambda _, e1, e2, e3, e4, e5, e6, e7, e8, e9, e10, e11, e12, e13, e14, e15, e16:
                        (
                            alpha[p, ..., t].reshape(-1)
                        *   np.array([e1, e2, e3, e4, e5, e6, e7, e8, e9, e10, e11, e12, e13, e14, e15, e16]).dot(layers.reshape((16, -1)))
                        *   integral[:, t]
                        ).reshape(-1),
            xdata = None,
            ydata = rho[p, ..., t].reshape(-1),
            p0    = epsilon.loc[p, t]
        )[0]

        for t in points[1:]:
            _epsilon.loc[p, t] = curve_fit(
                f     = lambda _, e1, e2, e3, e4, e5, e6, e7, e8, e9, e10, e11, e12, e13, e14, e15, e16:
                            (
                                alpha[p, ..., t].reshape(-1)
                            *   np.array([e1, e2, e3, e4, e5, e6, e7, e8, e9, e10, e11, e12, e13, e14, e15, e16]).dot(layers.reshape((16, -1)))
                            *   integral[:, t]
                            ).reshape(-1),
                xdata = None,
                ydata = rho[p, ..., t].reshape(-1),
                p0    = epsilon.loc[p, t]
            )[0]

        epsilon = _epsilon

    print("\tnu")
    previous_nus = nus.copy()
    for p in plates:
        integral = -(
                alpha[p].reshape((-1, n_points))
            *   epsilon.loc[p].dot(layers.reshape((16, -1))).T.values
            *   Nt[p].reshape((-1, n_points))
            ).cumsum(axis = 1)

        nus.loc[p] = curve_fit(
            f     = lambda _, nu:
                        (
                            alpha[p].reshape((-1, n_points))
                            *   epsilon.loc[p].dot(layers.reshape((16, -1))).T.values
                            *   np.exp(nu * integral)
                        ).reshape(-1),
            xdata = None,
            ydata = rho[p].reshape(-1),
            p0    = 1e-6
        )[0]
    
    print("\tr 0 & m")
    ts = np.tile(points, n_rows * n_columns)
    for p in plates:
        integral = np.exp(-(
                nus.loc[p, "nu"]
                *   alpha[p].reshape((-1, n_points))
                *   epsilon.loc[p].dot(layers.reshape((16, -1))).T.values
                *   Nt[p].reshape((-1, n_points))
            ).cumsum(axis = 1))
        
        ci = alphas.loc[p, "c i"].values.repeat(n_points)
        r0_m.loc[p] = curve_fit(
            f     = lambda _, r0, m:
                r0 * ci / (ci + np.exp(-m * ts)) * (epsilon.loc[p].dot(layers.reshape((16, -1))).T.values * integral).reshape(-1),
            xdata = 80085,
            ydata = rho[p].reshape(-1)
        )[0]
    
    print("\tc i")
    for p in plates:
        r0, m = r0_m.loc[p]
        for r, c in product(rows, columns):
            idx = (p, r, c)
            integral = np.exp(-nus.loc[p, "nu"] * alpha[idx] * epsilon.loc[p, f"epsilon {c2l[r, c]}"] * Nt[idx])
            
            c_i[idx] = curve_fit(
                f      = lambda _, ci:
                            r0 * ci / (ci + np.exp(-m * points)) * epsilon.loc[p, f"epsilon {c2l[r, c]}"] * integral,
                xdata  = 80085,
                ydata  = rho[idx],
                bounds = (0, np.inf)
            )[0]
    
    previous_alphas = alphas.copy()
    alphas.loc[:, "r0 i"] = list(r0_m["r0"].repeat(n_rows * n_columns))
    alphas.loc[:, "m i"] = list(r0_m["m"].repeat(n_rows * n_columns))
    alphas.loc[:, "c i"] = list(c_i)
    
    previous_alpha = alpha.copy()
    for idx in product(plates, rows, columns):
        r0, ci, mi = alphas.loc[idx]
        alpha[idx] = r0 * ci / (ci + np.exp(-mi * points))
    
    predictions = np.array([
            alpha[p].reshape((-1, n_points))
        *   epsilon.loc[p].dot(layers.reshape((16, -1))).T.values
        *   np.exp(-(
                nus.loc[p, "nu"]
            *   alpha[p].reshape((-1, n_points))
            *   epsilon.loc[p].dot(layers.reshape((16, -1))).T.values
            *   Nt[p].reshape((-1, n_points))
            ).cumsum(axis = 1))
        for p in plates
    ]).reshape((n_plates, n_rows, n_columns, n_points))
    
    current_score = lse(predictions.reshape(-1), rho.reshape(-1))
    if current_score < previous_score:
        previous_score = current_score
    else:
        alphas = previous_alphas
        alpha = previous_alpha
        epsilon = previous_epsilon
        nus = previous_nus
        break

Saving the prediction results :

In [ ]:
predictions = np.array([
        alpha[p].reshape((-1, n_points))
    *   epsilon.loc[p].dot(layers.reshape((16, -1))).T.values
    *   np.exp(-(
            nus.loc[p, "nu"]
        *   alpha[p].reshape((-1, n_points))
        *   epsilon.loc[p].dot(layers.reshape((16, -1))).T.values
        *   Nt[p].reshape((-1, n_points))
        ).cumsum(axis = 1))
    for p in plates
]).reshape((n_plates, n_rows, n_columns, n_points))

In [ ]:
np.save("predictions/level-2_alpha-epsilon_k-phi.npy", predictions)